In [1]:
from PIL import Image
from pathlib import Path
import argparse
from tqdm import tqdm
import glob
import os
import pandas as pd
from re import search

### Defining Paths of directories

In [11]:
path = '/home/ubuntu/bucket/projects'
projdir = '2019_07_11_JUMP-CP'
batchname = '2020_11_16_Scope1_PE'
platename = 'CPBroadPhenixNCP4__NC_BIN2_3P_2020-10-09T16_58_02-Measurement1'
#s3://imaging-platform/projects/2019_07_11_JUMP-CP/2020_11_16_Scope1_PE/images/CPBroadPhenixNCP4__NC_BIN2_3P_2020-10-09T16_58_02-Measurement1/Images/

batchpath =  os.path.join(path, projdir, batchname)

fpath = os.path.join(batchpath, 'images', platename, 'Images')

outpath = os.path.join(path, projdir, 'workspace', 'load_data.csv', batchname, platename)




fpath = [os.path.join(batchpath, 'images', f) for f in os.listdir(os.path.join(batchpath, 'images'))]

fimages = [os.path.join(i, "Images") for i in fpath]

# outpath = [os.path.join(path, projdir, 'workspace', 'load_data_csv', batchname, p) for p in plates]

plates = [i.split("/")[-1].split("__")[0] for i in fpath][0]

FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/bucket/projects/2019_07_11_JUMP-CP/2020_11_16_Scope1_PE/images'

### making output directory

In [10]:
def makedirectory():
    try:
        os.makedirs(outpath)
        print("Directory", outpath, "is created")
    except IOError:
        print("Directory", outpath, "already exists")
        
directory = makedirectory()


Directory /home/ubuntu/bucket/projects/2019_07_11_JUMP-CP/workspace/load_data.csv/2020_11_16_Scope1_PE/CPBroadPhenixNCP4__NC_BIN2_3P_2020-10-09T16_58_02-Measurement1 already exists


### Creating load data csvs

In [ ]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

def load_data():
    
    lst=[]

    for i, image in enumerate(os.listdir(fpath)):
        imgpath = fpath + "/" + image
        head, tail = os.path.split(imgpath)
        lst.append(tail)
   
    

    ch0 = [s for s in lst if "_ch_0" in s]
    ch1 = [s for s in lst if "_ch_1" in s]
    ch2 = [s for s in lst if "_ch_2" in s]
    ch3 = [s for s in lst if "_ch_3" in s]

    zippedlist = list(zip(ch0, ch1, ch2, ch3))


    df = pd.DataFrame(zippedlist, columns=['FileName_OrigDNA', 
                                        'FileName_OrigProtein', 
                                        'FileName_OrigMito',
                                       'FileName_OrigER'])

    df['PathName_OrigDNA'] = fpath
    df['PathName_OrigProtein'] = fpath
    df['PathName_OrigMito'] = fpath
    df['PathName_OrigER'] = fpath
    df['Metadata_Plate'] = platename
    df['Metadata_Well'] = (df['FileName_OrigDNA']
                       .str.split('_')
                       .str.get(5)
                       .str.split('-')
                       .str.get(0)
                       .apply(lambda x: x[0] + str(0) + x[1] if len(x) < 3 else x)
                          )
                           
    df['Metadata_Site'] = (df.FileName_OrigDNA
                       .str.split('_')
                       .str.get(6)
                       .str.split('.')
                       .str.get(0))


    colnames = ['FileName_OrigDNA', 'PathName_OrigDNA',
             'FileName_OrigProtein', 'PathName_OrigProtein',
             'FileName_OrigMito', 'PathName_OrigMito',
             'FileName_OrigER', 'PathName_OrigER', 'Metadata_Plate', 'Metadata_Well', 'Metadata_Site']

    return df.reindex(columns=colnames)

loaddata = load_data()

loaddata.to_csv(outpath + "/" + 'load_data.csv')


### Creating load_data_with_illum csv

In [ ]:
def load_data_with_illum():
    
    lst=[]

    for i, image in enumerate(sorted(os.listdir(fpath))):
        imgpath = fpath + "/" + image
        head, tail = os.path.split(imgpath)
        img_name = os.path.splitext(tail)[0]
        lst.append(tail)
    

    ch0 = [s for s in lst if "_ch_0" in s]
    ch1 = [s for s in lst if "_ch_1" in s]
    ch2 = [s for s in lst if "_ch_2" in s]
    ch3 = [s for s in lst if "_ch_3" in s]

    zippedlist = list(zip(ch0, ch1, ch2, ch3))


    df = pd.DataFrame(zippedlist, columns=['FileName_OrigDNA', 
                                        'FileName_OrigProtein', 
                                        'FileName_OrigMito',
                                       'FileName_OrigER'])

    df['PathName_OrigDNA'] = fpath
    df['PathName_OrigProtein'] = fpath
    df['PathName_OrigMito'] = fpath
    df['PathName_OrigER'] = fpath
    df['Metadata_Plate'] = platename
    df['FileName_IllumDNA'] = platename+'_IllumDNA.npy'
    df['PathName_IllumDNA'] = batchpath + '/' + 'illum' + '/' + platename
    df['FileName_IllumProtein'] = platename+'_IllumProtein.npy'
    df['PathName_IllumProtein'] = batchpath + '/' + 'illum' + '/' + platename
    df['FileName_IllumMito'] = platename+'_IllumMito.npy'
    df['PathName_IllumMito'] = batchpath + '/' + 'illum' + '/' + platename
    df['FileName_IllumER'] = platename+'_IllumER.npy'
    df['PathName_IllumER'] = batchpath + '/' + 'illum' + '/' + platename
    
    df['Metadata_Well'] = (df.FileName_OrigDNA
                       .str.split('_')
                       .str.get(5)
                       .str.split('-')
                       .str.get(0)
                       .apply(lambda x: x[0] + str(0) + x[1] if len(x) < 3 else x)
                          )
    

                           
    df['Metadata_Site'] = (df.FileName_OrigDNA
                       .str.split('_')
                       .str.get(6)
                       .str.split('.')
                       .str.get(0)
                          )


    colnames = ['FileName_OrigDNA', 'PathName_OrigDNA',
             'FileName_OrigProtein', 'PathName_OrigProtein',
             'FileName_OrigMito', 'PathName_OrigMito',
             'FileName_OrigER', 'PathName_OrigER', 
             'Metadata_Plate','Metadata_Well', 
            'Metadata_Site', 'FileName_IllumDNA', 'PathName_IllumDNA', 
            'FileName_IllumProtein', 'PathName_IllumProtein', 
            'FileName_IllumMito', 'PathName_IllumMito','FileName_IllumER','PathName_IllumER']

    return df.reindex(columns=colnames)

loaddataillum = load_data_with_illum()
loaddataillum.to_csv(outpath + "/" + 'load_data_with_illum.csv')